In [1]:
# 1️⃣ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2️⃣ Navigate to the folder containing your notebook
%cd /content/drive/MyDrive/Colab\ Notebooks/

# 3️⃣ Initialize git (if not already)
!git init

# 4️⃣ Connect to your GitHub repo
!git remote add origin https://github.com/PhkhakadzeJumber/deep-learning-final.git

# 5️⃣ Set git user (only needs to be done once per Colab runtime)
!git config --global user.name "PhkhakadzeJumber"
!git config --global user.email "jubaphkhakadze@gmail.com"

# 6️⃣ Add your notebook
!git add data_training_final.ipynb

# 7️⃣ Commit changes
!git commit -m "Add final training notebook"

# 8️⃣ Push to GitHub
!git branch -M main
!git push -u origin main



Mounted at /content/drive
